# sklearn.pipeline.Pipeline
Pipeline é uma simples maneira de manter o nosso código de pré-processamento e modelagem de dados organizado. Para ser mais específico, pipelines agrupam as etapas de pré-processamento e modelagem como se fosse uma única etapa.



De acordo com a documentação oficial, disponível em [sklearn.pipeline.Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html):

    Pipeline of transforms with a final estimator

Para abordarmos a definição disponível na documento oficial temos que, primeiramente, estar cientes que Scikit-learn basicamente apresenta duas interfaces:
1. __Transform__ é utilizado para o pré-processamento de dados, e apresenta os seguintes métodos:<br>
1.1. __fit__<br>
1.2. __transform__<br>
2. __Estimator__ é utilizado para a modelagem de dados, e apresenta os seguintes métodos:<br>
2.1. __fit__<br>
2.2. __predict__

Portanto, podemos traduzir a documentação oficial como:

    Pipeline de transformações (pré-processamentos) com um estimator (modelo) final.

## Base de dados
Analise o DataFrame abaixo e observe que possuímos colunas com dados numéricos e categóricos. Portanto, teremos que realizar o pré-processamento de todas as colunas.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Base de dados
data = pd.read_csv('data.csv')

# Seleção das colunas features e target 
X = data.drop(columns=['Price'])
y = data['Price']

# Divisão dos dados em treinamento e validações
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [2]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.8272,4217.0


## Passo 1: Definição dos Pré-processamentos
* Para as colunas com variáveis numéricas, iremos preencher os valores faltantes com o valor mais frequente da coluna;
* Para as colunas com variáveis categóricas, iremos preencher os valores faltantes com o valor mais frequente da coluna e em seguida converteremos as variáveis categóricas em numéricas através da classe [sklearn.preprocessing.OrdinalEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html).

Portanto teremos dois objetos Pipelines.


In [3]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder

# Pré-processamento para os dados numéricos
numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))
])

# Pré-processamento para os dados categóricos
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OrdinalEncoder())
])

### sklearn.compose.ColumnTransformer
Como estamos trabalhando com dois objetos pipelines, devemos uní-los e especificar em quais colunas cada Pipeline será aplicado. Portanto, vamos conhecer a classe __ColumnTranformer__.

De acordo com a [documentação oficial](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), um objeto ColumnTransformer __aplica__ os __transformers__ em um array ou em um __DataFrame__.

In [4]:
# Seleciona as colunas categóricas
categorical_cols = [column_name for column_name in X_train if X_train[column_name].dtype == 'object']

# Seleciona as colunas numéricas
numerical_cols = [column_name for column_name in X_train if X_train[column_name].dtype in ['int64', 'float64']]

In [5]:
from sklearn.compose import ColumnTransformer

# Une os pré-processamentos dos dados numéricos e categóricos e define em quais colunas deverão ser aplicados
all_transformers = ColumnTransformer(
    transformers=[
        ('numerical_transformer', numerical_pipeline, numerical_cols),
        ('categorical_transformer', categorical_pipeline, categorical_cols)
    ])

## Passo 2: Declaração do Modelo
Agora vamos definir um modelo Random Forest com a classe RandomForestRegressor.

In [6]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

## Paso 3: Criação do Pipeline
Neste último passo, unimos todos os passos de pré-processamento (all_transformers) e o modelo RandomForestRegressor em um único objeto Pipeline.

Utilizaremos uma técnica mais prática para a criação do Pipeline através do método [__make_pipeline()__](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html).

Em seguida utilizamos o método fit() e predict(). Observe que:
* [predict()](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline.predict) aplica automaticamente o pré-processamento nos dados de validação antes de realizar as predições _(Apply transforms to the data, and predict with the final estimator)_.

In [7]:
from sklearn.pipeline import make_pipeline

# Une os códigos de pré-processamentos e o modelo em um único pipeline
pipeline = make_pipeline(all_transformers, model)

# Pré-processamento dos dados de treinamento e aprendizagem do modelo
pipeline.fit(X_train, y_train)

# Pré-processamento automática dos dados de validação e geração das predições
predicoes = pipeline.predict(X_valid)
predicoes

array([1695497.  ,  863207.  ,  598340.  , ..., 1258360.  , 1281738.88,
       1055620.  ])